In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
install.packages("pastecs")
install.packages("plyr")
install.packages("dplyr")
install.packages("ggplot2")
install.packages("tidyverse")
install.packages("data.table")
install.packages("corrplot")
install.packages("plotly")
install.packages("heatmaply")
install.packages("ggcorrplot")
install.packages("factoextra")
install.packages("vegan")


In [ ]:
devtools::install_github("laresbernardo/lares")

In [ ]:
library(pastecs)
library(plyr)
library(dplyr)
library(ggplot2)
library(tidyverse)
library(data.table)
library(plotly)
library(heatmaply)
library(ggcorrplot)
library(factoextra)
library(gplots)
library(cluster)
library(vegan)
library(useful)
library(RColorBrewer)
library(corrplot)
library(lares)

In [ ]:
# EDA of (train_features = TF)

# Part 1

# call dataset
train_features = read.csv("../input/lish-moa/train_features.csv", na.strings = c("","NA"), stringsAsFactors = F, header = T, check.names = FALSE)

In [ ]:
#replace "-" by "_" for attributes
train_features <- train_features %>% rename_with(~ tolower(gsub("-", "_", .x, fixed = TRUE)))
head(train_features)

In [ ]:
str(train_features)

In [ ]:
# number of rows and columns
dim(train_features)

In [ ]:
# number of missing values
sum(is.na(train_features))

In [ ]:
#integer attributes
ICL <- train_features %>% select_if(is.numeric)

In [ ]:
#integer attributes of gene expressions and cell viability
ICL <- ICL[,-1]
head(ICL)

In [ ]:
#name of integer attributes
names(ICL)

In [ ]:
#number of integer attributes
length(ICL)

In [ ]:
#Statistics Description of integer attributes
ICLS <- round(stat.desc(ICL),2)
ICLS <- as.data.frame(ICLS)
ICLS

In [ ]:
# part 2

# find quantiles of integer attributes
y <- apply(ICL,2,quantile)[c(2,4),]
# rename rows
rownames(y) <- c("Q1","Q3")
# calculate IQR for all integer attributes
IQR <- y[2,]-y[1,]
#merge IQR to y
y <- rbind(y,IQR)
#calculate lower limit and upper limit
LL <- y[1,]-(1.5*y[3,])
UL <- y[2,]+(1.5*y[3,])
#merge lower limit and upper limit to y
y <- rbind(y,LL,UL)
#merge these table to statistics description
ICLS <- rbind(ICLS,y)
ICLS

In [ ]:
#calculate Outliers
Outliers <- c()
for (x in (1:length(ICL))){
    Outliers <- c(Outliers,length(ICL[(ICL[,x] < ICLS[18,x]) | (ICL[,1] > ICLS[19,x]),x]))
   }

In [ ]:
#merge Outliers
ICLS <- round(rbind(ICLS,"Outliers" = Outliers),2)
ICLS

In [ ]:
# select sample size of 5000 rows for Shapiro-Wilk’s Normality Test
s1 <- sample(seq_len(nrow(ICL)), 5000, FALSE)
s1 <- ICL[s1,]
s1

In [ ]:
# apply Shapiro-Wilk’s Normality Test
Normality_p_value <- c()
Not_Normal <- c()
for (x in (1:length(s1))){
                           Z <- shapiro.test(s1[,x])
                            n <- Z$p.value
                            if (n > 0.05) {
                                                   Not_Normal <- c(Not_Normal, colnames(s1)[,x])
                                                }
                            Normality_p_value = c(Normality_p_value, Z$p.value)
   }

In [ ]:
#features that are not normal
Not_Normal

In [ ]:
#merge p.values of Normality Test
ICLS <- round(rbind(ICLS,"Normality_p_value" = Normality_p_value),2)
ICLS

In [ ]:
#convert first row to column values
ICLS <- setDT(ICLS, keep.rownames = TRUE)[]
colnames(ICLS)[1] <- "Statistics"
ICLS <- as.data.frame(ICLS)
ICLS

In [ ]:
# overall statistics of gene features
ICLSgene <- ICLS %>% select(starts_with("g_"))
GeneMin <- min(ICLSgene[4,]) ; 
GeneMax <- max(ICLSgene[5,]) ; 
GeneMedian <- median(as.numeric(as.vector(ICLSgene[8,]))) ; 
GeneMean <- mean(as.numeric(as.vector(ICLSgene[9,])))
GeneSE.mean <- mean(as.numeric(as.vector(ICLSgene[10,])))

GeneMin ; GeneMax ; GeneMedian ; GeneMean ; GeneSE.mean

In [ ]:
# overall statistics of cell features
ICLScell <- ICLS %>% select(starts_with("c_"))
CellMin <- min(ICLScell[4,]) ; 
CellMax <- max(ICLScell[5,]) ; 
CellMedian <- median(as.numeric(as.vector(ICLScell[8,]))) ; 
CellMean <- mean(as.numeric(as.vector(ICLScell[9,])))
CellSE.mean <- mean(as.numeric(as.vector(ICLScell[10,])))

CellMin ; CellMax ; CellMedian ; CellMean ; CellSE.mean

In [ ]:
#Distribution of randomly selected gene features
dataG <- ICL %>% select(starts_with("g_"))
data2G <- stack(dataG[,c(sample(1:772, 50, replace=FALSE))])
colnames(data2G)[2] <- "Gene_Features"
ggplot(data2G, aes(x=Gene_Features, y=values, fill=Gene_Features)) + geom_boxplot()


In [ ]:
#Distribution of randomly selected cell features
dataC <- ICL %>% select(starts_with("C_"))
data2C <- stack(dataC[,c(sample(1:100, 50, replace=FALSE))])
colnames(data2C)[2] <- "Cell_Features"
ggplot(data2C, aes(x=Cell_Features, y=values, fill=Cell_Features)) + geom_boxplot()

In [ ]:
#Which numerical attributes seem to be correlated? 
# Compute a correlation matrix
ICL_corr <- cor(ICL)
ICL_corr

In [ ]:
# minimum value of correlation coefficent
min((ICL_corr))

In [ ]:
# maximum value of correlation coefficent
max((ICL_corr))

In [ ]:
# Compute a matrix of correlation p-values
p.mat <- cor_pmat(ICL,)
# convert matrix to upper half
p.mat[lower.tri(p.mat,diag=TRUE)] <- 0

In [ ]:
p.mat

In [ ]:
correlation_coefficient <- c()
p_value = c()
fea1 <- c()
fea2 <- c()
for (i in 1:872){
    for (j in 1:872) {
                    x <- p.mat[i,j]
                    y <- ICL_corr[i,j]
                    if (x <= 0.05 & x != 0 ){
                                   correlation_coefficient <- c(correlation_coefficient, y)
                                   p_value <- c(p_value,x)
                                   fea1 <- c(fea1,rownames(p.mat)[i])
                                   fea2 <- c(fea2,colnames(p.mat)[j])
                                    }
                    }
                }

In [ ]:
# dataframe of significant correlated features = sig_cor_fea_df
sig_cor_fea_df <- data.frame(fea1, fea2, correlation_coefficient, p_value)
dim(sig_cor_fea_df)
head(sig_cor_fea_df)

In [ ]:
# minimum value of correlation coefficient of among significantly correlated relations.
min(abs(sig_cor_fea_df$correlation_coefficient))

In [ ]:
# minimum value of correlation coefficient of among significantly correlated relations.
max(abs(sig_cor_fea_df$correlation_coefficient))

In [ ]:
# plot correlation graph for significantly related features
# significantly related features are

In [ ]:
#frequency of fea1
FRQUENCYfea1df <- count(sig_cor_fea_df, fea1)
FRQUENCYfea1df <- FRQUENCYfea1df[order(FRQUENCYfea1df$n ,decreasing = TRUE),]
colnames(FRQUENCYfea1df)[2] <- "count1"
dim(FRQUENCYfea1df)
head(FRQUENCYfea1df)

In [ ]:
#frequency of fea2
FRQUENCYfea2df <- count(sig_cor_fea_df, fea2)
FRQUENCYfea2df <- FRQUENCYfea2df[order(FRQUENCYfea2df$n ,decreasing = TRUE),]
colnames(FRQUENCYfea2df)[2] <- "count2"
dim(FRQUENCYfea2df)
head(FRQUENCYfea2df)

In [ ]:
# gather top 20 fea1 and fea2 with higheest frequency
# and plot correlation graph
#Top 20 of fea1
F1 <- as.character(FRQUENCYfea1df[1:20,1])
F1
#Top 20 of fea2
F2 <- as.character(FRQUENCYfea2df[1:20,1])
F2
F <- c(F1,F2)
F

In [ ]:
#extract features from original dataset based on F1 and F2
ICLF <- ICL[,F]
dim(ICLF)
head(ICLF)

In [ ]:
corrplot(cor(ICLF), method="color")

In [ ]:
#add frequency count to sig_cor_fea_df
sig_cor_fea_fre_df <- merge(sig_cor_fea_df, FRQUENCYfea1df, by = "fea1")
sig_cor_fea_fre_df <- merge(sig_cor_fea_fre_df, FRQUENCYfea2df, by = "fea2")

In [ ]:
#count total frequency
sig_cor_fea_fre_df$total <- sig_cor_fea_fre_df$count1 + sig_cor_fea_fre_df$count2
sig_cor_fea_fre_df <- sig_cor_fea_fre_df[order(sig_cor_fea_fre_df$total ,decreasing = TRUE),]
dim(sig_cor_fea_fre_df)
head(sig_cor_fea_fre_df)

In [ ]:
# extract top 100 of total 
SEM <- sig_cor_fea_fre_df[1:100,c(1,2)]
x1 <- levels( factor( SEM$fea2))
x2 <- levels( factor( SEM$fea1))
length(x1) ; length(x2)
X <- c(as.character(x1), as.character(x2))
X <- sort(X)

In [ ]:
#extract features from original dataset based on x1 and x2
ICLX <- ICL[,X]
dim(ICLX)
head(ICLX)

In [ ]:
corrplot(cor(ICLX), method="color")

In [ ]:
# PCA on gene features
gene <- train_features %>% select(starts_with("g_"))

genepca <- prcomp(gene, center = TRUE, scale. = TRUE)

In [ ]:
summary(genepca)

PC1 accounts for 22.4% varaince and PC2 accounts for 4.39% variance
For 95% of cumulative variance, first 521 PCs can be selected

In [ ]:
get_eig(genepca)

In [ ]:
fviz_eig(genepca, title = "Variance", ncp = 20)

In [ ]:
# gene features contribution to PC1
fviz_contrib(genepca, choice = "var", axes = 1, top = 20)

For Dim1,many different features have a similar strength of contribution. The order of these feature names doesn’t necessarily reveal anything interesting

In [ ]:
# gene features contribution to PC2
fviz_contrib(genepca, choice = "var", axes = 2, top = 20)

#In PC2, “g-173” leading the pack. However, PC2 contributes less than 5% of the overall variance

In [ ]:
# Visualization of PC1 and PC2 by type of treatments
fviz_pca_ind(genepca, label = "none",
             habillage = train_features %>% mutate(cp_type = if_else(cp_type == "trt_cp", "Treatment compound", "Treatment control")) %>% pull(cp_type),
             # habillage = train_features$cp_type,
             alpha.ind = 0.3,
             palette = c("#ffc5cf", "black"),
             title = "Treatment type") + theme(legend.position = "top")

The control treatments show much tighter clustering in the first two PCs than the real compound treatments. 

In [ ]:
# Visualization of PC1 and PC2 by type of dose
fviz_pca_ind(genepca, label = "none",
             habillage = train_features$cp_dose,
             alpha.ind = 0.3,
             palette = c("#ffc5cf", "black"),
             title = "Type of Dose") + theme(legend.position = "top")

distributions for Dose D1 vs Dose D2 are closely matched.

In [ ]:
# Visualization of PC1 and PC2 by Treatment Duration
fviz_pca_ind(genepca, label = "none",,
             habillage = train_features$cp_time,
             alpha.ind = 0.3,
             palette = c("#ffc5cf", "black", "#cfffc5"),
             title = "Treatment Time") + theme(legend.position = "top")

In [ ]:
# PCA on Cell features
cell <- train_features %>% select(starts_with("c_"))

cellpca <- prcomp(cell, center = TRUE, scale. = TRUE)
cellpca

In [ ]:
summary(cellpca)

for 95% of cumulative variance, we need select first 44 PCs 

In [ ]:
get_eig(cellpca)

In [ ]:
fviz_eig(cellpca, title = "Variance", ncp = 20)

PC1 accounts for 83.75% of variance, and PC2 accounts for 1.19% variance

In [ ]:
# cell features contribution to PC1
fviz_contrib(cellpca, choice = "var", axes = 1, top = 20)

we can see that contribution of cell features to PC1 is almost identical

In [ ]:
# gene features contribution to PC2
fviz_contrib(cellpca, choice = "var", axes = 2, top = 20)

we can see that contribution of cell features to PC2 declines from left to right

In [ ]:
# Visualization of cell-PC1 and cell-PC2 by type of treatments
fviz_pca_ind(cellpca, label = "none",
             habillage = train_features %>% mutate(cp_type = if_else(cp_type == "trt_cp", "Treatment compound", "Treatment control")) %>% pull(cp_type),
             # habillage = train_features$cp_type,
             alpha.ind = 0.3,
             palette = c("#ffc5cf", "black"),
             title = "Treatment type") + theme(legend.position = "top")

# Visualization of cell-PC1 and cell-PC2 by type of dose
fviz_pca_ind(cellpca, label = "none",
             habillage = train_features$cp_dose,
             alpha.ind = 0.3,
             palette = c("#ffc5cf", "black"),
             title = "Type of Dose") + theme(legend.position = "top")

# Visualization of cell-PC1 and cell-PC2 by Treatment Duration
fviz_pca_ind(cellpca, label = "none",,
             habillage = train_features$cp_time,
             alpha.ind = 0.3,
             palette = c("#ffc5cf", "black", "#cfffc5"),
             title = "Treatment Time") + theme(legend.position = "top")

we can see that distribution of compound treatment is more compact than control treatment
Distribution of D1 and D2 are closely matched.
There is small cluster of Time-24 above the horizontal line and cluster of Time-72 below horizontal lines that are standout, otherwise rest of the distriubtions of all three times are closely matched.